# Comparative Analysis of Neighborhoods | Canadian Rent Data Preparation

The Canada Mortgage and Housing Corporation is Canada's national housing agency. It is responsible for providing reliable housing market data and analysis. We will retrieve 2023 canadian median rent prices.

Link: https://www.cmhc-schl.gc.ca/professionnels/marche-du-logement-donnees-et-recherche/donnees-sur-le-logement/tableaux-de-donnees/donnees-sur-le-marche-locatif/enquete-sur-les-logements-locatifs-centres-urbains-loyers-moyens

## [1] Working environment set up

Before starting, we need to install and import libraries.

In [1]:
# Data Storage and File Handling
import json
!pip install openpyxl

# Data Manipulation and Processing
!pip install pandas
import pandas as pd
import re
import unicodedata
!pip install unidecode
import unidecode
!pip install fuzzywuzzy
!pip install python-Levenshtein
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Geolocation and Mapping
!pip install geocoder
import geocoder
!pip install geopy
from geopy.distance import geodesic

# Miscellaneous
import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

print("Libraries imported.")

Libraries imported.


## [2] Data Collection

Before starting, we need to open the venue dataframe and the geolocation dictionary.

In [11]:
# Load from CSV file
cities_df = pd.read_csv('venue_df_output.csv', encoding='utf-8')

In [2]:
# Load from JSON file
with open("geolocation_dic_output.json", "r") as f:
    citiesinfo_dic = json.load(f)

Then, to retrieve the Canadian housing data, we will investigate the zip folder provided by the Canadian statistics department to how the tabs are formatted.

In [3]:
# Load the CSV file to see its structure and content
cmhc_path = r'C:\Users\marin\OneDrive\Documents\GITHUB\urban-rental-market-survey-data-average-rents-urban-centres-2023-en.xlsx'

# Load the Excel file to get all sheet names
excel_file = pd.ExcelFile(cmhc_path)

In [4]:
# Function to find the first valid header row
def find_valid_header_test1(df):
    for i, row in df.iterrows():
        print(f"Testing row {i}: {row.tolist()}")                               # Print the current row being tested
        if all(isinstance(val, str) and val.strip() != "" for val in row):      # Check if the row contains valid column names (non-empty strings)
            print(f"Valid header found at row {i}")                             # Print when a valid header is found
            return i, row.tolist()                                              # Return the row index and the column names
    return None, None                                                           # Return None if no valid header is found

# Iterate the function over each sheet
for sheet in excel_file.sheet_names:
    df = pd.read_excel(excel_file, sheet, header = None)                        # Load without headers
    print(f"Sheet Name: {sheet}")
    sheet_title = df.iloc[0, 0] if not pd.isnull(df.iloc[0, 0]) else 'No title' # The sheet title is assumed to be in the first row
    print(f"Sheet Title: {sheet_title}")
    header_index, columns = find_valid_header_test1(df)                               # Try to find a valid header starting from row 4
    print("\n")

Sheet Name: CSD
Sheet Title: Private Row (Townhouse) and Apartment Average Rent by Bedroom Type and Census Subdivision for Centres 10,000+
Testing row 0: ['Private Row (Townhouse) and Apartment Average Rent by Bedroom Type and Census Subdivision for Centres 10,000+', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
Testing row 1: ['October 2023', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
Testing row 2: [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
Testing row 3: ['Province', 'Centre', 'Census Subdivision', 'Dwelling Type', 'Bachelor', nan, '1 Bedroom', nan, '2 Bedroom', nan, '3 Bedroom\n+', nan, 'Total', nan]
Testing row 4: ['Nfld.Lab.', "St. John's", 'Holyrood (T)', 'Row', '--', nan, '--', nan, '--', nan, '--', nan, '--', nan]
Testing row 5: ['Nfld.Lab.', "St. John's", 'Holyrood (T)', 'Apt & Other', '--', nan, '--', nan, '--', nan, '--', nan, '--', nan]
Testing row 6: ['Nfld.Lab.', "St. John's", 'Holyrood (T)', 'Total', '

We ran an automated header checker to evaluate each row of the worksheets and identify the correct header row. However, we noticed that the process needs some refinement. The checker actually selects rows containing additional information, which aren’t the official headers, but rather supplementary data. A clearer definition of what qualifies as a header is necessary to ensure accurate detection.

In [5]:
# Function to find the first valid header row
def find_valid_header_test2(df):
    for i, row in df.iterrows():
        print(f"Testing row {i}: {row.tolist()}")                                   # Print the current row being tested
        valid_strings = [isinstance(val, str) and val.strip() != "" for val in row] # Create a mask of whether each value is a valid string
        consecutive_count = 0
        for is_valid in valid_strings:                                              
            if is_valid:
                consecutive_count += 1
                if consecutive_count >= 3:                                          # If we find 3 consecutive valid strings, this is the header row
                    print(f"Valid header found at row {i}")                         # Print when a valid header is found
                    return i, [val for val in row if pd.notna(val)]                 # Return the row index and the column names (removing NaN)
            else:
                consecutive_count = 0                                               # Reset the counter if interrupted by NaN or invalid string    
    return None, None                                                               # Return None if no valid header is found

# Iterate the function over each sheet
for sheet in excel_file.sheet_names:
    df = pd.read_excel(excel_file, sheet, header = None)                            # Load without headers
    print(f"Sheet Name: {sheet}")
    sheet_title = df.iloc[0, 0] if not pd.isnull(df.iloc[0, 0]) else 'No title'     # The sheet title is assumed to be in the first row
    print(f"Sheet Title: {sheet_title}")
    header_index, columns = find_valid_header_test2(df)                                   # Try to find a valid header starting from row 4
    print("\n")

Sheet Name: CSD
Sheet Title: Private Row (Townhouse) and Apartment Average Rent by Bedroom Type and Census Subdivision for Centres 10,000+
Testing row 0: ['Private Row (Townhouse) and Apartment Average Rent by Bedroom Type and Census Subdivision for Centres 10,000+', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
Testing row 1: ['October 2023', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
Testing row 2: [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
Testing row 3: ['Province', 'Centre', 'Census Subdivision', 'Dwelling Type', 'Bachelor', nan, '1 Bedroom', nan, '2 Bedroom', nan, '3 Bedroom\n+', nan, 'Total', nan]
Valid header found at row 3


Sheet Name: Neighbourhood
Sheet Title: Private Row (Townhouse) and Apartment Average Rent by Bedroom Type and Neighbourhood for Census Metropolitan Areas
Testing row 0: ['Private Row (Townhouse) and Apartment Average Rent by Bedroom Type and Neighbourhood for Census Metropolitan Areas', n

Great, it worked! we can now focus on finding the correct worksheet with the neighborhood cut to review its columns fields.

In [6]:
# Function to find the first valid header row
def find_valid_header(df):
    for i, row in df.iterrows():
        valid_strings = [isinstance(val, str) and val.strip() != "" for val in row]
        consecutive_count = 0
        for is_valid in valid_strings:
            if is_valid:
                consecutive_count += 1
                if consecutive_count >= 3:
                    return i, [val for val in row if pd.notna(val)]
            else:
                consecutive_count = 0                                                  
    return None, None

# Dictionary to store columns of each sheet
sheetsdetails_dic = {}

# Build a function to remove the '\n' from the columns names
def clean_columns(column_list):
    return [col.replace('\n', ' ') if isinstance(col, str) else col for col in column_list]

# Iterate over each sheet to search for valid columns
for sheet in excel_file.sheet_names:
    df = pd.read_excel(excel_file, sheet_name = sheet, header = None)
    sheet_title = df.iloc[0, 0] if not pd.isnull(df.iloc[0, 0]) else 'No title'
    header_index, columns = find_valid_header(df) 
    # When valid columns are found, update the dictionary
    if columns:
        cleaned_columns = clean_columns(columns)
        sheetsdetails_dic[sheet] = {'sheet_title': sheet_title, 'header_index': header_index, 'columns': columns, 'cleaned_columns': cleaned_columns}

# Print the found columns for each sheet
for sheet_name, sheet_info in sheetsdetails_dic.items():
    neighborhood_sheet_name = sheet_name
    if "Neighbourhood" in sheet_info['columns']: # Quartier means neighborhood in French
        print(f"Sheet Name: {sheet_name}")
        print(f"Sheet Title: {sheet_info['sheet_title']}")
        print(f"Header Index: {sheet_info['header_index']}")
        print(f"Columns: {sheet_info['cleaned_columns']}")
        print("\n")
        break  

Sheet Name: Neighbourhood
Sheet Title: Private Row (Townhouse) and Apartment Average Rent by Bedroom Type and Neighbourhood for Census Metropolitan Areas
Header Index: 3
Columns: ['Province', 'Centre', 'Zone', 'Neighbourhood', 'Dwelling Type', 'Bachelor', '1 Bedroom', '2 Bedroom', '3 Bedroom +', 'Total']




In [7]:
# Load the correct sheet using the header index to build the DataFrame
df = pd.read_excel(excel_file, sheet_name=neighborhood_sheet_name, header=sheetsdetails_dic[neighborhood_sheet_name]['header_index'])

# Select only the columns from sheet_info['columns']
selected_columns = sheetsdetails_dic[neighborhood_sheet_name]['columns']
cmhc_df = df[selected_columns]
cmhc_df.dropna(inplace=True)
print(cmhc_df.shape)
cmhc_df.head()

(3915, 10)


,Province,Centre,Zone,Neighbourhood,Dwelling\nType,Bachelor,1\nBedroom,2\nBedroom,3 Bedroom\n+,Total
0,Nfld.Lab.,St. John's,St. John's East,St. John's East,Row,**,**,**,**,$1352
1,Nfld.Lab.,St. John's,St. John's East,St. John's East,Apt & Other,$695,$987,$1264,$998,$1117
2,Nfld.Lab.,St. John's,St. John's East,St. John's East,Total,$695,$984,$1262,$1238,$1123
3,Nfld.Lab.,St. John's,St. John's East,Total,Row,**,**,**,**,$1352
4,Nfld.Lab.,St. John's,St. John's East,Total,Apt &\nOther,$695,$987,$1264,$998,$1117


## [3] Data Cleaning and Formatting

To be able to use the data from the canadian housing institude cmhc, we will need to format their summary table into pure data. We will have to remove the rows without data and rows with totals. Then we will need to filter only on the canadian cities within our exercise. Though we are facing another language barrier here as the french files includes french naming conventions and french accents. Hence to be able to compare both data, we will have to normalize the data by removing accents and converting to lowercase.

In [9]:
# Clean up the column names
cmhc_df.columns = cmhc_df.columns.str.replace('\n', ' ')

# Remove the extra rows
cmhc_df = cmhc_df[cmhc_df['Dwelling Type'] == 'Total']
cmhc_df = cmhc_df[~cmhc_df['Neighbourhood'].str.contains('Total', case=False, na=False)]

# Remove the empty data rows
rent_columns = cmhc_df.columns[-5:-1]  # Adjust the range to exclude the last column named Total
cmhc_df[rent_columns] = cmhc_df[rent_columns].replace(r'[\$,]', '', regex=True).apply(pd.to_numeric, errors = 'coerce')
cmhc_df = cmhc_df.dropna(subset = rent_columns, how = 'all')

# Build a function to normalise city names for comparison (removing accents, 'city', etc.)
def normalize_city_name(city):
    city = unicodedata.normalize('NFKD', city).encode('ASCII', 'ignore').decode('utf-8')
    city = re.sub(r'\b(city|ville)\b', '', city, flags=re.IGNORECASE).strip()
    city = re.sub(r'[^\w\s]', '', city).lower()
    return city

citiesinfo_original_list = [city for city in citiesinfo_dic['city0']]
citiesinfo_normalized_list = [normalize_city_name(city) for city in citiesinfo_original_list]
cmhc_df['Centre_normalized'] = cmhc_df['Centre'].apply(normalize_city_name)

# Get the list of retrieved cities with their normalized name
citiesretrieved_normalized_list = cmhc_df[cmhc_df['Centre_normalized'].isin(citiesinfo_normalized_list)]['Centre_normalized'].unique().tolist()

# Get the list of retrieved cities with their original name according to our dictionary formatting
citiesretrieved_original_list = []
for normalized_city in citiesretrieved_normalized_list:
    if normalized_city in citiesinfo_normalized_list:
        # Find the index of the normalized city and retrieve the original city name from the original list
        original_city = citiesinfo_original_list[citiesinfo_normalized_list.index(normalized_city)]
        citiesretrieved_original_list.append(original_city)
        
# Get the list of missing cities with their original name according to our dictionary formatting
citiesmissing_original_list = set(citiesinfo_original_list) - set(citiesretrieved_original_list)
citiesmissing_original_list = list(citiesmissing_original_list)

# Display the results
print(f"Retrieved cities from wiki_dic in the dataframe: {citiesretrieved_original_list}")
print(f"Missing cities from wiki_dic that were not found in the dataframe: {citiesmissing_original_list}")

# Final list of cities to investigate in the cmhc file
canadiancity_list = cmhc_df[cmhc_df['Centre_normalized'].isin(citiesinfo_normalized_list)]['Centre'].unique().tolist()
cmhc_df = cmhc_df[cmhc_df['Centre'].isin(canadiancity_list)]

# Update the city name with the one from our citiesinfo dictionary
normalized_to_original_dic = dict(zip(citiesinfo_normalized_list, citiesinfo_original_list))
cmhc_df['City'] = cmhc_df['Centre_normalized'].map(normalized_to_original_dic).fillna(cmhc_df['Centre'])

# Display the final filtered dataframe
print(cmhc_df.shape)
cmhc_df.head()

Retrieved cities from wiki_dic in the dataframe: ['Montreal', 'Quebec City', 'Ottawa', 'Toronto', 'Vancouver']
Missing cities from wiki_dic that were not found in the dataframe: ['Paris']
(337, 12)


,Province,Centre,Zone,Neighbourhood,Dwelling Type,Bachelor,1 Bedroom,2 Bedroom,3 Bedroom +,Total,Centre_normalized,City
368,Que,Montréal,Downtown Montréal/Îles-des-Soeurs,Ville-Marie,Total,1146.0,1429.0,2012.0,2072.0,1527.0,montreal,Montreal
371,Que,Montréal,Downtown Montréal/Îles-des-Soeurs,Île-des-Soeurs,Total,927.0,1139.0,1357.0,1419.0,1226.0,montreal,Montreal
377,Que,Montréal,Sud-Ouest/Verdun,Verdun,Total,800.0,952.0,938.0,1394.0,1028.0,montreal,Montreal
380,Que,Montréal,Sud-Ouest/Verdun,South West,Total,NaN,974.0,1246.0,NaN,1154.0,montreal,Montreal
386,Que,Montréal,LaSalle,LaSalle,Total,750.0,924.0,1199.0,1596.0,1273.0,montreal,Montreal


Paris is obviously missing from the matching cities as we are working on the canadian file.

We aim to identify the most effective method for matching neighborhoods in the CMHC dataset with those in the cities_df dataframe. To do this, we will evaluate three techniques: direct matching, fuzzy matching, and geolocation matching. By comparing the accuracy of each method, we will determine which approach provides the highest number of successful matches.

In [12]:
# Build a function to normalize the neighborhood names: Remove accents, convert to lowercase, strip spaces
def normalize_neighborhood(name):
    if pd.notna(name):
        name = unidecode.unidecode(name).lower().strip()
        return name
    return name

# Apply normalization
cmhc_df['Neigh_normalized'] = cmhc_df['Neighbourhood'].apply(normalize_neighborhood)
cities_df['Neigh_normalized'] = cities_df['Neighborhood'].apply(normalize_neighborhood)

**First Matching Method: Direct Matching**

This method compares neighborhood names exactly as they are written in both datasets. It relies on the assumption that the names in both sources are spelled identically, including spaces, punctuation, and capitalization. While simple and efficient, this method can miss matches due to minor variations like accents, abbreviations, or typos.

In [13]:
def direct_match(first_neigh, first_city, second_neigh, second_city):
    if city != second_city:
        return False
    second_parts = [part.strip() for part in second_neigh.split(',')]  # Split second_parts by commas
    return any(part in first_neigh for part in second_parts)  # Return True if any part matches first_neigh

**Second Matching Method: Fuzzy Matching**

This method uses algorithms to compare the similarity of neighborhood names, allowing for partial matches even when names aren't identical. It applies techniques such as token sorting and Levenshtein distance to find close matches based on the degree of similarity between strings. This method is more flexible than direct matching but can occasionally result in false positives or less relevant matches.

In [14]:
# Prepare city neighborhood lists for fuzzy matching
cities_neigh_dict = {}
# Group the DataFrame by 'City' and then split and process neighborhoods within each city
for city, group in cities_df.groupby('City'):
    neigh_list = [part.strip() for neigh in group['Neigh_normalized'].unique() for part in neigh.split(',')]
    cities_neigh_dict[city] = neigh_list

def fuzzy_match(first_neigh, first_city, threshold=70):
    best_match = None
    highest_score = 0
    match, score = process.extractOne(first_neigh, cities_neigh_dict[first_city], scorer=fuzz.token_sort_ratio)
    if score >= threshold:
        best_match = match
    return best_match if best_match else None

In [15]:
# Initialize a cache to store geolocation results
geo_cache = {}

def geoloc_match(location, city, max_retries=5, max_distance_km=2):
    # Adapt the city name to the geocoder format
    city = citiesinfo_dic['city1'][citiesinfo_dic['city0'].index(city)]
    # Check if the location and city are already in the cache
    if (location, city) in geo_cache:
        # Check if a postal code match was successful
        return geo_cache[(location, city)]['postalcode'] is not None

    retries = 0
    lati_long_coords = None
    
    # Try to retrieve the coordinates with retries
    while lati_long_coords is None and retries < max_retries:
        g = geocoder.arcgis('{}, {}'.format(location, city))
        lati_long_coords = g.latlng
        if lati_long_coords is None:
            retries += 1
    
    # If geolocation fails after retries, cache it as None and return False
    if lati_long_coords is None:
        geo_cache[(location, city)] = {'coordinates': None, 'postalcode': None}
        return False
    
    # If geolocation is successful, check proximity with citysource_df
    lat, lon = lati_long_coords
    min_distance = float('inf')
    closest_postalcode = None
    
    for _, cities_row in cities_df.iterrows():
        cities_coords = (cities_row['Latitude'], cities_row['Longitude'])
        neighborhood_coords = (lat, lon)
        distance = geodesic(cities_coords, neighborhood_coords).kilometers
        
        if distance < min_distance:
            min_distance = distance
            closest_postalcode = cities_row['Postalcode']
    
    # If the minimum distance is greater than max_distance_km, consider it a failed match
    if min_distance > max_distance_km:
        closest_postalcode = None
    
    # Cache the result (whether or not a postal code was matched)
    geo_cache[(location, city)] = {'coordinates': lati_long_coords, 'postalcode': closest_postalcode}

    # Return True if there was a successful match, otherwise return False
    return closest_postalcode is not None

**Third Matching Method: Geolocation Matching**

This method works by converting neighborhood names into latitude and longitude coordinates and finding the closest match based on geographical proximity. This method uses the actual physical locations of neighborhoods to determine matches, which is useful when names are inconsistent or ambiguous. However, it depends on accurate geolocation data and may take longer to process due to external API requests.

**Testing the Matching Methods**

Let's test our 3 methods.

In [16]:
# Initialize count for matched neighborhoods using both methods
direct_match_count = 0
fuzzy_match_count = 0
geoloc_match_count = 0
cmhc_totalcount = len(cmhc_df)  # Total number of neighborhoods in cmhc_df

# Iterate through cmhc_df and check for matches using each method
for _, cmhc_row in cmhc_df.iterrows():

    # Direct match search
    for _, cities_row in cities_df.iterrows():
        if direct_match(cmhc_row['Neigh_normalized'], cmhc_row['City'], cities_row['Neigh_normalized'], cities_row['City']):
            direct_match_count += 1
            break  # Stop after the first direct match for each cmhc_df row
    
    # Fuzzy match search
    if fuzzy_match(cmhc_row['Neigh_normalized'], cmhc_row['City']):
        fuzzy_match_count += 1

    # Geolocation match search
    if geoloc_match(cmhc_row['Neigh_normalized'], cmhc_row['City']):
        geoloc_match_count += 1

# Calculate total matches and percentages
direct_match_total = direct_match_count
fuzzy_match_total = fuzzy_match_count
geoloc_match_total = geoloc_match_count

if cmhc_totalcount > 0:
    direct_match_percentage = (direct_match_total / cmhc_totalcount) * 100
    fuzzy_match_percentage = (fuzzy_match_total / cmhc_totalcount) * 100
    geoloc_match_percentage = (geoloc_match_total / cmhc_totalcount) * 100
else:
    direct_match_percentage = 0
    fuzzy_match_percentage = 0
    geoloc_match_percentage = 0

# Determine the best matching method
if direct_match_total >= fuzzy_match_total and direct_match_total >= geoloc_match_total:
    best_method = "Direct Matching"
elif fuzzy_match_total >= direct_match_total and fuzzy_match_total >= geoloc_match_total:
    best_method = "Fuzzy Matching"
else:
    best_method = "Geolocation Matching"

# Output the results
print(f"Total neighborhoods to match: {cmhc_totalcount}")
print(f"Number of direct matches: {direct_match_total} ({direct_match_percentage:.2f}%)")
print(f"Number of fuzzy matches: {fuzzy_match_total} ({fuzzy_match_percentage:.2f}%)")
print(f"Number of geolocation matches: {geoloc_match_total} ({geoloc_match_percentage:.2f}%)")
print(f"Best matching method: {best_method}")

Total neighborhoods to match: 337
Number of direct matches: 40 (11.87%)
Number of fuzzy matches: 157 (46.59%)
Number of geolocation matches: 280 (83.09%)
Best matching method: Geolocation Matching


The geolocation method is the one with the best matching score. Let's apply it to connect our cmhc data to the cities_df postal coldes. To maximise our chances of connecting our neighborhoods together, instead of looking for the postal code that is the closest to our geolocation, we will list all the postal codes in a radius of 2km respective to our location in the cmhc file. Each of them will be impact by the rent data.

In [17]:
# Create a function to geolocate each neighborhood in cmhc_df
geo_cache = {}

def get_neigh_latilong(location, city, max_retries=5):
    city_full = citiesinfo_dic['city1'][citiesinfo_dic['city0'].index(city)]
    if (location, city_full) in geo_cache:
        return geo_cache[(location, city_full)]
    retries = 0
    lati_long_coords = None
    while lati_long_coords is None and retries < max_retries:
        g = geocoder.arcgis('{}, {}'.format(location, city_full))
        lati_long_coords = g.latlng
        if lati_long_coords is None:
            retries += 1
    if lati_long_coords is None:
        lati_long_coords = [None, None]  # Set as None if the geolocation fails
    geo_cache[(location, city_full)] = lati_long_coords
    return lati_long_coords

# Create a function to calculate distance and find the closest match
def find_postalcodes(lat, lon, city, max_distance_km):
    nearby_postalcodes = []
    city_rows_df = cities_df[cities_df['City'] == city]
    for _, city_row in city_rows_df.iterrows():
        city_coords = (city_row['Latitude'], city_row['Longitude'])
        neighborhood_coords = (lat, lon)
        distance = geodesic(city_coords, neighborhood_coords).kilometers
        if distance <= max_distance_km:
            nearby_postalcodes.append(city_row['Postalcode'])
    return nearby_postalcodes

In [18]:
# Keep only the specified columns
canadianrentN_df = cmhc_df[['City', 'Neighbourhood', 'Bachelor', '1 Bedroom', '2 Bedroom', '3 Bedroom +']]
canadianrentB_df = cmhc_df[['City', 'Zone', 'Bachelor', '1 Bedroom', '2 Bedroom', '3 Bedroom +']]

# Rename the columns according to the number of bedrooms
rent_columns = ['Median Rent Studio', 'Median Rent 1 Bedroom', 'Median Rent 2 Bedrooms', 'Median Rent 3 Bedrooms']
canadianrentN_df.columns = ['City','Neighborhood'] + rent_columns
canadianrentB_df.columns = ['City','Borough'] + rent_columns

In [19]:
# Apply geolocation to each neighborhood and find the postal codes near enough the geolocation
canadianrentN_df['LatLong'] = canadianrentN_df.apply(lambda row: get_neigh_latilong(row['Neighborhood'], row['City']), axis=1)
canadianrentN_df[['Latitude', 'Longitude']] = pd.DataFrame(canadianrentN_df['LatLong'].tolist(), index = canadianrentN_df.index)

# Find the postal codes within less than 2 kilometers of the cmch neighborhoods geolocation
canadianrentN_df = canadianrentN_df.dropna(subset=['Latitude', 'Longitude'])
canadianrentN_df['Postal_Code'] = canadianrentN_df.apply(lambda row: find_postalcodes(row['Latitude'], row['Longitude'], row['City'], max_distance_km=2), axis=1)

# Filter out rows without nearby postal codes, expand lists of postal codes into separate rows
canadianrentN_df = canadianrentN_df.dropna(subset=['Postal_Code'])
canadianrentN_df['Postal_Code'] = canadianrentN_df['Postal_Code'].apply(lambda x: x if isinstance(x, list) else [])
canadianrentN_df = canadianrentN_df.explode('Postal_Code').reset_index(drop=True)

# Keep only 'City', 'Postal_Code', and rent columns to finalize the dataframe
canadianrentN_df = canadianrentN_df[['City', 'Postal_Code'] + rent_columns]
print(canadianrentN_df.shape)
canadianrentN_df.head()

(1409, 6)


,City,Postal_Code,Median Rent Studio,Median Rent 1 Bedroom,Median Rent 2 Bedrooms,Median Rent 3 Bedrooms
0,Montreal,H3A,1146.0,1429.0,2012.0,2072.0
1,Montreal,H5A,1146.0,1429.0,2012.0,2072.0
2,Montreal,H3B,1146.0,1429.0,2012.0,2072.0
3,Montreal,H5B,1146.0,1429.0,2012.0,2072.0
4,Montreal,H3C,1146.0,1429.0,2012.0,2072.0


To make sure we won't be missing a significant amount of median rent data in our main dataframe, we will also compute median rents based on boroughs from the cmhc source and match them with the corresponding boroughs in our dataset to fill in the missing values after the neighborhoods matching.

In [20]:
# Apply geolocation to each borough and find the postal codes near enough the geolocation
canadianrentB_df['LatLong'] = canadianrentB_df.apply(lambda row: get_neigh_latilong(row['Borough'], row['City']), axis=1)
canadianrentB_df[['Latitude', 'Longitude']] = pd.DataFrame(canadianrentB_df['LatLong'].tolist(), index=canadianrentB_df.index)

# Find the postal codes within less than 5 kilometers of the cmch borough geolocation
canadianrentB_df = canadianrentB_df.dropna(subset=['Latitude', 'Longitude'])
canadianrentB_df['Postal_Code'] = canadianrentB_df.apply(lambda row: find_postalcodes(row['Latitude'], row['Longitude'], row['City'], max_distance_km=5), axis=1)

# Filter out rows without nearby postal codes, expand lists of postal codes into separate rows
canadianrentB_df = canadianrentB_df.dropna(subset=['Postal_Code'])
canadianrentB_df['Postal_Code'] = canadianrentB_df['Postal_Code'].apply(lambda x: x if isinstance(x, list) else [])
canadianrentB_df = canadianrentB_df.explode('Postal_Code').reset_index(drop=True)

# Keep only 'City', 'Postal_Code', and rent columns to finalize the dataframe
canadianrentB_df = canadianrentB_df[['City', 'Postal_Code'] + rent_columns]
print(canadianrentB_df.shape)
canadianrentB_df.head()

(6614, 6)


,City,Postal_Code,Median Rent Studio,Median Rent 1 Bedroom,Median Rent 2 Bedrooms,Median Rent 3 Bedrooms
0,Montreal,H5A,1146.0,1429.0,2012.0,2072.0
1,Montreal,H3B,1146.0,1429.0,2012.0,2072.0
2,Montreal,H3C,1146.0,1429.0,2012.0,2072.0
3,Montreal,H4C,1146.0,1429.0,2012.0,2072.0
4,Montreal,H3E,1146.0,1429.0,2012.0,2072.0


In this process, we filter on the data we want to connect later to our main dataframe and we fill on the missing gaps. To do so, we first focus on calculating average rents based on postal codes for different property types (studio, 1-bedroom, 2-bedroom, and 3-bedroom units). For rows with complete data (no missing values), we calculate "factors" for each rent type by dividing the rent for each unit type by the overall average rent for that row. These factors are then averaged across all valid rows to create a set of reference factors. For rows with missing rent data, we estimate the missing values using the row-specific average (based on available rent data) and the predefined factors. Finally, the missing values are filled in and the data is rounded for clarity.

In [21]:
# Ensure rent columns are numeric
canadianrentN_df[rent_columns] = canadianrentN_df[rent_columns].apply(pd.to_numeric, errors = 'coerce')
canadianrentB_df[rent_columns] = canadianrentB_df[rent_columns].apply(pd.to_numeric, errors = 'coerce')

# Group by specified columns and calculate mean for rent columns
canadianrentN_df = canadianrentN_df.groupby(['City', 'Postal_Code'], as_index = False).mean()
canadianrentB_df = canadianrentB_df.groupby(['City', 'Postal_Code'], as_index = False).mean()

# Define a helper function to compute average factors
def calculate_avg_factors(df):    
    # Drop rows with NaN values in any rent columns
    df_copy = df.copy()
    df_copy = df_copy.dropna(subset = rent_columns)
    
    # Calculate average rent for each row
    df_copy['Average Rent'] = df_copy[rent_columns].mean(axis=1)
    
    # Compute factors and average factors for each city
    avg_factors = {}
    for city in df_copy['City'].unique():
        city_df_copy = df_copy[df_copy['City'] == city]
        avg_factors[city] = {f"{rent_col}": (city_df_copy[rent_col] / city_df_copy['Average Rent']).mean() 
                             for rent_col in rent_columns}
    return avg_factors
    
# Calculate the average factors for both Neighborhood and Borough DataFrames
avgN_factors = calculate_avg_factors(canadianrentN_df)
avgB_factors = calculate_avg_factors(canadianrentB_df)

# Print the results
print("Neighborhood Average Factors:")
for city, factors in avgN_factors.items():
    print(f"Average factors for {city}:")
    for factor_name, average in factors.items():
        print(f"  {factor_name}: {average:.2f}")

Neighborhood Average Factors:
Average factors for Montreal:
  Median Rent Studio: 0.71
  Median Rent 1 Bedroom: 0.94
  Median Rent 2 Bedrooms: 1.07
  Median Rent 3 Bedrooms: 1.28
Average factors for Ottawa:
  Median Rent Studio: 0.73
  Median Rent 1 Bedroom: 0.88
  Median Rent 2 Bedrooms: 1.09
  Median Rent 3 Bedrooms: 1.29
Average factors for Quebec City:
  Median Rent Studio: 0.79
  Median Rent 1 Bedroom: 0.95
  Median Rent 2 Bedrooms: 1.07
  Median Rent 3 Bedrooms: 1.20
Average factors for Toronto:
  Median Rent Studio: 0.74
  Median Rent 1 Bedroom: 0.89
  Median Rent 2 Bedrooms: 1.08
  Median Rent 3 Bedrooms: 1.29
Average factors for Vancouver:
  Median Rent Studio: 0.74
  Median Rent 1 Bedroom: 0.85
  Median Rent 2 Bedrooms: 1.14
  Median Rent 3 Bedrooms: 1.27


In [22]:
# Define the function to compute row-specific averages and fill missing values
def fill_missing_rents_row_based(row, city_avg_factors):
    # Calculate row-specific averages based on available rents
    available_rent_averages = [row[col] / city_avg_factors[col] for col in rent_columns if pd.notna(row[col])]
    # Calculate the mean of the available row-specific averages
    if available_rent_averages:
        row_avg = sum(available_rent_averages) / len(available_rent_averages)
        # Fill missing rent values using the row-specific average and factors
        for col in rent_columns:
            if pd.isna(row[col]):
                row[col] = row_avg * city_avg_factors[col]
    return row

# Main function to apply filling process for each DataFrame
def fill_missing_rents(df, avg_factors):
    df = df.apply(lambda row: fill_missing_rents_row_based(row, avg_factors[row['City']]), axis=1)
    df[rent_columns] = df[rent_columns].round(0)
    return df.reset_index(drop=True)

# Apply to both DataFrames
canadianrentN_df = fill_missing_rents(canadianrentN_df, avgN_factors)
canadianrentB_df = fill_missing_rents(canadianrentB_df, avgB_factors)

In [23]:
# Merge the neighborhoods median rents into the main dataframe 
cities_df = cities_df.merge(canadianrentN_df.drop(columns='City'), how='left', left_on='Postalcode', right_on='Postal_Code').drop(columns='Postal_Code')

# Fill the missing median rents from the main dataframe with the borough median rents
for column in rent_columns:
    if column in canadianrentB_df.columns:
        # Merge only the specific rent column from `canadianrentB_df`
        merged_column = cities_df[['Postalcode', column]].set_index('Postalcode').combine_first(canadianrentB_df.set_index('Postal_Code')[[column]])
        # Update only NaN values in cities_df using merged_column
        cities_df[column] = cities_df[column].fillna(merged_column[column])

## [4] Saving

In [24]:
# Save the DataFrame to a CSV file with UTF-8 encoding
cities_df.to_csv('canadianrent_df_output.csv', index=False, encoding='utf-8')
print("DataFrame saved as 'canadianrent_df_output.csv'.")

DataFrame saved as 'canadianrent_df_output.csv'.
